In [9]:
pip install -upgrade pip


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [10]:
# Install opencv (using the correct package name)
!pip install opencv-python

# Import necessary modules (assuming open_code is a custom function)
try:
  from open_code import open_code
except ImportError:
  print("The 'open_code' function is not found. Make sure it's installed or defined.")

# You can now use opencv and open_code (if available) in your code

The 'open_code' function is not found. Make sure it's installed or defined.


In [11]:
pip install opencv-contrib-python

In [12]:
import cv2
print(cv2.__version__)

4.11.0


In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np

In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/wildfire-prediction-dataset


In [15]:
train_dir = r"/kaggle/input/wildfire-prediction-dataset/train"
valid_dir = r"/kaggle/input/wildfire-prediction-dataset/valid"
test_dir =r"/kaggle/input/wildfire-prediction-dataset/test"

#set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')

Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [16]:
model=Sequential([Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
                  MaxPooling2D(pool_size=(2,2)),
                  Conv2D(64, (3,3), activation='relu'),
                  MaxPooling2D(pool_size=(2,2)),
                  Flatten(),
                  Dense(128, activation='relu'),
                  Dropout(0.5),
                  Dense(1, activation='sigmoid')])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [18]:
history = model.fit(train_generator, epochs=3, validation_data=valid_generator, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
946/946 ━━━━━━━━━━━━━━━━━━━━ 239s 250ms/step - accuracy: 0.8874 - loss: 0.2727 - val_accuracy: 0.8130 - val_loss: 0.4115
Epoch 2/3
946/946 ━━━━━━━━━━━━━━━━━━━━ 205s 216ms/step - accuracy: 0.9303 - loss: 0.1880 - val_accuracy: 0.9457 - val_loss: 0.1456
Epoch 3/3
946/946 ━━━━━━━━━━━━━━━━━━━━ 208s 219ms/step - accuracy: 0.9398 - loss: 0.1653 - val_accuracy: 0.9546 - val_loss: 0.1303


In [19]:
model.save('fire_model.h5')

In [20]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

model=load_model('fire_model.h5')
print("Model loaded successfully")

Model loaded successfully


In [21]:
!pip install gradio

In [22]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

model = load_model('fire_model.h5')

def predict_image(img):
    img = img.resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    result = "Wildfire" if prediction > 0.5 else "No Wildfire"
    return result

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),  # Updated input definition
    outputs="text",
    title="Forest Fire Detection",
    description="Upload an image to predict whether it contains a wildfire."
)


In [23]:
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://09d2e37aa9214919be.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
